# UNFILTERED 
# Deduplicating Common-Crawl for LLM Training Datasets. 

In this notebook, we will cover how to perform the minhash deduplication algorithm on the super-massive common crawl dataset.

Common crawl is an index of public webpages famously used to train GPT-3. LLMs require massive amounts of data to train on and early foundation models like GPT-3 and T5 saw improvements in model performance due to deduplication efforts. Deduplication makes it far less likely that the model regurgitates memorized text leading to better responses.

*See [Deduplicating Training Data Makes Language Models Better (Lee et. all)](https://aclanthology.org/2022.acl-long.577.pdf)*

---

## The MinHash Deduplication algorithm

If you google "minhash deduplication" you'll find a variety of sources that can walk you through the aglorithm. [Finding Near Duplicates with Jaccard Similarity and MinHash by Nelson Elhage](https://blog.nelhage.com/post/fuzzy-dedup/) is a great place to start, but if you are looking for the canonical reference for the MinHash deduplication algorithm, it originates from the seminal paper by Andrei Z. Broder, published in 1997, titled:

```text
"On the resemblance and containment of documents"
Published in: Proceedings of the Compression and Complexity of Sequences 1997 (SEQUENCES '97)
Publisher: IEEE Computer Society
DOI: 10.1109/SEQUEN.1997.666900
```

A video walkthough of the algorithm is also available through [Mike Mull's presentation on YouTube](https://www.youtube.com/watch?v=KKNPmvELUP4). He even provides a [jupyter notebook](https://github.com/papers-we-love/san-diego/blob/master/presentations/2016-11-03-resemblance-containment-documents/Broder97.ipynb) detailing the core primatives and how they are calculated in pure python. 


## Introducing the Minhash Deduplication for Common Crawl 




## Authenticating with AWS to access S3

Crawl data is free to access by anyone from anywhere. The data is hosted by Amazon Web Services’ Open Data Sets Sponsorships program on the bucket s3://commoncrawl/, located in the US-East-1 (Northern Virginia) AWS Region. The most performative means of accessing Common crawl is through s3, so you'll need to authenticate with an `AWS_ACCESS_KEY_ID` or `AWS_SECRET_ACCESS_KEY`. Common Crawl data can also be accessed without authentication, anonymously via it's http endpoint. 


In [ ]:
import daft
from daft.io import IOConfig, S3Config
import os
from dotenv import load_dotenv

# Make sure to define your AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY in your environment variables or in a .env file
load_dotenv()

s3_config = S3Config(
    region_name="us-east-1",
    requester_pays=True,
    key_id=os.environ["AWS_ACCESS_KEY_ID"],
    access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    anonymous=False,
)

IO_CONFIG = IOConfig(s3=s3_config)
daft.set_planning_config(default_io_config=IO_CONFIG)

DaftContext(_ctx=<builtins.PyDaftContext object at 0x11086e750>)

## Loading Common Crawl Documents 
The Common Crawl corpus contains petabytes of data, with it's oldest entries dating back to 2008. It contains raw web page data, metadata extracts, and text extracts.

We will be accessing Common Crawl through WARC files since daft supports the format natively with `daft.read_warc(uri)`

In [43]:
NUM_ROWS = 100

In [74]:
df_warc = daft.read_warc("s3://commoncrawl/crawl-data/CC-MAIN-2018-17/segments/*/warc/*.warc.gz").limit(NUM_ROWS)
df_warc.show() # Inspect the first 8 rows

WARC-Record-IDUtf8,WARC-TypeUtf8,"WARC-DateTimestamp(Nanoseconds, Some(""Etc/UTC""))",Content-LengthInt64,WARC-Identified-Payload-TypeUtf8,warc_contentBinary,warc_headersUtf8
caae53dc-d718-4852-9d43-b0d7881c9740,warcinfo,2018-04-24 15:49:11 UTC,324,None,"b""robots: classic\r\nhostname: ip-10""...","{""Content-Type"":""application/warc-fields"",""WARC-Filename"":""CC-MAIN-20180424154911-20180424174911-00000.warc.gz""}"
9e2e0a42-4674-4d68-bdba-aa50c2eb9612,request,2018-04-24 16:16:34 UTC,293,None,"b""GET /company/zapchasti/708-avtozap""...","{""Content-Type"":""application/http; msgtype=request"",""WARC-IP-Address"":""88.214.230.62"",""WARC-Target-URI"":""http://00043.ru/company/zapchasti/708-avtozap-inomar-samara"",""WARC-Warcinfo-ID"":""<urn:uuid:caae53dc-d718-4852-9d43-b0d7881c9740>""}"
998a8186-f2f5-43e8-a46d-9259350322c0,response,2018-04-24 16:16:34 UTC,39851,application/xhtml+xml,"b""HTTP/1.1 200 OK\r\nServer: Apache\r""...","{""Content-Type"":""application/http; msgtype=response"",""WARC-Block-Digest"":""sha1:LGG4RZM7S4B7ABHHFISVJ3FTAHRQBROG"",""WARC-Concurrent-To"":""<urn:uuid:9e2e0a42-4674-4d68-bdba-aa50c2eb9612>"",""WARC-IP-Address"":""88.214.230.62"",""WARC-Payload-Digest"":""sha1:NX57X344QQKCCB7BMBOK4WQAE4QUP66V"",""WARC-Target-URI"":""http://00043.ru/company/zapchasti/708-avtozap-inomar-samara"",""WARC-Warcinfo-ID"":""<urn:uuid:caae53dc-d718-4852-9d43-b0d7881c9740>""}"
8504336e-6718-45da-9d40-9c73629d0513,metadata,2018-04-24 16:16:34 UTC,20,None,"b""fetchTimeMs: 570\r\n\r\n""","{""Content-Type"":""application/warc-fields"",""WARC-Concurrent-To"":""<urn:uuid:998a8186-f2f5-43e8-a46d-9259350322c0>"",""WARC-Target-URI"":""http://00043.ru/company/zapchasti/708-avtozap-inomar-samara"",""WARC-Warcinfo-ID"":""<urn:uuid:caae53dc-d718-4852-9d43-b0d7881c9740>""}"
a1485c32-677c-4677-918d-5b7642ba7cf8,request,2018-04-24 16:36:15 UTC,301,None,"b""GET /2012/01/todays-loot-26th-janu""...","{""Content-Type"":""application/http; msgtype=request"",""WARC-IP-Address"":""172.217.5.225"",""WARC-Target-URI"":""http://00tn.blogspot.com/2012/01/todays-loot-26th-january-2011.html"",""WARC-Warcinfo-ID"":""<urn:uuid:caae53dc-d718-4852-9d43-b0d7881c9740>""}"
71b09b3d-5229-4798-b3c9-e8b782ecc317,response,2018-04-24 16:36:15 UTC,207309,application/xhtml+xml,"b""HTTP/1.0 200 OK\r\nServer: GSE\r\n""...","{""Content-Type"":""application/http; msgtype=response"",""WARC-Block-Digest"":""sha1:BCO7WCU6JCMAK5IZPG5UUWRMCSLMEVJV"",""WARC-Concurrent-To"":""<urn:uuid:a1485c32-677c-4677-918d-5b7642ba7cf8>"",""WARC-IP-Address"":""172.217.5.225"",""WARC-Payload-Digest"":""sha1:QFLXAB4IDUJXC5ZYDD75AJRNXSNNOXWK"",""WARC-Target-URI"":""http://00tn.blogspot.com/2012/01/todays-loot-26th-january-2011.html"",""WARC-Warcinfo-ID"":""<urn:uuid:caae53dc-d718-4852-9d43-b0d7881c9740>""}"
6178fb97-cbd3-4029-a400-42ef7fc0ec0a,metadata,2018-04-24 16:36:15 UTC,20,None,"b""fetchTimeMs: 379\r\n\r\n""","{""Content-Type"":""application/warc-fields"",""WARC-Concurrent-To"":""<urn:uuid:71b09b3d-5229-4798-b3c9-e8b782ecc317>"",""WARC-Target-URI"":""http://00tn.blogspot.com/2012/01/todays-loot-26th-january-2011.html"",""WARC-Warcinfo-ID"":""<urn:uuid:caae53dc-d718-4852-9d43-b0d7881c9740>""}"
5099ae2b-8994-4c3f-aa8f-0885e120de69,request,2018-04-24 16:12:53 UTC,211,None,"b""GET / HTTP/1.0\r\nHost: 00yc0.lifo""...","{""Content-Type"":""application/http; msgtype=request"",""WARC-IP-Address"":""104.217.203.113"",""WARC-Target-URI"":""http://00yc0.lifotaiji.com/"",""WARC-Warcinfo-ID"":""<urn:uuid:caae53dc-d718-4852-9d43-b0d7881c9740>""}"


In [ ]:
# Lets investigate the different types of payloads we have: 
df_warc.select("WARC-Identified-Payload-Type").distinct().show()

WARC-Identified-Payload-TypeUtf8
text/plain
image/jpeg
text/html
None
application/xhtml+xml


In [95]:
from daft import col

# For the purposes of this notebook, lets focus on text/plain and text/html payloads
df_html = df_warc.where(col("WARC-Identified-Payload-Type")== "text/html")

@daft.func()
def remove_http_headers(x: str) -> str:
    if x is None:
        return ""
    if len(x.split("\r\n\r\n")) > 1:
        return x.split("\r\n\r\n")[1]
    return ""

# We also need to seperate the http headers from the payloads
df_html = df_html.with_column("content_raw", 
    remove_http_headers(col("warc_content").try_decode("utf-8"))
).where(col("content_raw") != "")

# Then we will simplify the dataframe to just the content and the WARC-Record-ID
df_html = df_html.with_columns_renamed({
    "WARC-Identified-Payload-Type": "content_type",
    "WARC-Record-ID": "id",
}).select("id", "content_type", "content_raw")

df_html.show(3)


idUtf8,content_typeUtf8,content_rawUtf8
96963309-6d80-4743-9401-d50faf2f7d90,text/html,"<!DOCTYPE html> <!--[if IE 7]><html class=""ie7 no-js"" lang=""en-US"" xmlns:og=""//opengraphprotocol.org/schema/"" xmlns:fb=""//www.facebook.com/2008/fbml""<![endif]--> <!--[if lte IE 8]><html class=""ie8 no-js"" lang=""en-US"" xmlns:og=""//opengraphprotocol.org/schema/"" xmlns:fb=""//www.facebook.com/2008/fbml""<![endif]--> <!--[if (gte IE 9)|!(IE)]><!--><html class=""not-ie no-js"" lang=""en-US"" xmlns:og=""//opengraphprotocol.org/schema/"" xmlns:fb=""//www.facebook.com/2008/fbml""><!--<![endif]-->"
7772d841-60c2-4fd1-be96-6b704f6c36b8,text/html,"<!DOCTYPE html> <!--[if IE 7]><html class=""ie7 no-js"" lang=""en-US"" xmlns:og=""//opengraphprotocol.org/schema/"" xmlns:fb=""//www.facebook.com/2008/fbml""<![endif]--> <!--[if lte IE 8]><html class=""ie8 no-js"" lang=""en-US"" xmlns:og=""//opengraphprotocol.org/schema/"" xmlns:fb=""//www.facebook.com/2008/fbml""<![endif]--> <!--[if (gte IE 9)|!(IE)]><!--><html class=""not-ie no-js"" lang=""en-US"" xmlns:og=""//opengraphprotocol.org/schema/"" xmlns:fb=""//www.facebook.com/2008/fbml""><!--<![endif]-->"
1815f93d-8b84-481f-b6d4-82f1add145f7,text/html,"<!DOCTYPE html> <html lang=""de-DE"" class=""no-js""> <head itemscope=""itemscope"" itemtype=""http://schema.org/WebSite""> <meta charset=""UTF-8"" /> <meta name=""viewport"" content=""width=device-width, initial-scale=1"" /> <link rel=""profile"" href=""http://gmpg.org/xfn/11"" />"


In [70]:
html_body = df_html.select("content_raw").where(col("id") == "4ce3bec4-0cd6-4858-aa1f-ff59a49e68fe")
example = html_body.to_pydict()["content_raw"][0]

DaftCoreException: DaftError::FieldNotFound Column col(id) not found.

### Extracting Text from HTML body

In [97]:
from selectolax.parser import HTMLParser
import re
import hashlib

# Define a UDF to extract text from HTML content, Specifically (article, main, p, h1, h2, h3, li)
@daft.func()
def extract_blocks(html: str) -> list[str]:
    tree = HTMLParser(html)
    for n in tree.css("script,style,noscript"):  n.decompose()
    blocks = []
    for node in tree.css("article, main, p, h1, h2, h3, li"):
        txt = node.text(separator=" ", strip=True)
        if not txt: 
            continue
        blocks.append(txt)
    return blocks

df_html = df_html.with_column("content_extracted", extract_blocks(col("content_raw")))
df_html.show()

In [ ]:
df_prep = df_warc.with_column("word_tokens", 
    col("warc_content").try_decode("utf-8").str.normalize(
        remove_punct=True, 
        lowercase=True, 
        nfd_unicode=True, 
        white_space=True
    ).str.replace(r"\d","0",True
    ).str.split(" ") 
)

df_prep.show(3)


The Minhash Deduplication algorithm



In [ ]:
K = 5          # shingle size 
M = 64         # number of hash permutations
R = 8          # rows per band
B = M // R     # bands

## Word K grams
with 64 bit int shingle

In [22]:
# UDF: generate k-word shingles as 64-bit integers
import re, hashlib
from daft import lit

def k_shingles(tokens: list[str], k: int = 5) -> list[int]:
    if not tokens:
        return []
    norm = [t for t in tokens if t]
    if len(norm) < k:
        return []
    seen = set()
    out: list[int] = []
    for i in range(len(norm) - k + 1):
        gram = " ".join(norm[i:i+k])
        h = hashlib.sha1(gram.encode()).digest()[:8]
        v = int.from_bytes(h, "big")
        if v not in seen:
            seen.add(v)
            out.append(v)
    return out

udf_kshingles = daft.udf(k_shingles, return_dtype=daft.DataType.list(daft.DataType.int64()))

# apply UDF

df_shingles = df_tokens.with_column(
    "shingles",
    udf_kshingles(col("tokens"), lit(K))
)

df_shingles.select("doc_id", "shingles").show(3)



TypeError: udf() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given

In [ ]:
# Minhash signature computation
M = 64

import random
random.seed(42)

# Pre-generate M (a, b) pairs for hash functions over 64-bit space
A = [random.randrange(1, 2**61-1) for _ in range(M)]
B = [random.randrange(0, 2**61-1) for _ in range(M)]
P = (1 << 61) - 1  # a large prime (2^61-1 is prime)
MASK64 = (1 << 64) - 1

from typing import List

def minhash_signature(shingles: List[int], A: List[int], B: List[int], P: int) -> List[int]:
    if not shingles:
        return [MASK64] * len(A)
    sig = []
    for a, b in zip(A, B):
        m = MASK64
        for s in shingles:
            # universal hash on 64-bit shingles, then fold to 64-bit
            h = ((a * s + b) % P) & MASK64
            if h < m:
                m = h
        sig.append(m)
    return sig

udf_minhash = daft.udf(
    minhash_signature,
    return_dtype=daft.DataType.list(daft.DataType.int64())
)

df_sig = df_shingles.with_column(
    "signature",
    udf_minhash(col("shingles"), lit(A), lit(B), lit(P))
)

df_sig.select("doc_id", col("signature").list.length().alias("M"), "signature").show(3)



In [ ]:
# LSH banding and candidate generation
R = 8
BANDS = M // R

from typing import List, Tuple

def band_signature(signature: List[int], rows: int, bands: int) -> List[Tuple[int, int]]:
    # returns list of (band_index, band_hash)
    if not signature:
        return []
    out: List[Tuple[int, int]] = []
    for b in range(bands):
        start = b * rows
        end = start + rows
        if end > len(signature):
            break
        # hash the slice into a 64-bit bucket
        h = 1469598103934665603  # FNV offset
        for v in signature[start:end]:
            h ^= (v & 0xFFFFFFFFFFFFFFFF)
            h = (h * 1099511628211) & 0xFFFFFFFFFFFFFFFF
        out.append((b, h))
    return out

udf_band = daft.udf(
    band_signature,
    return_dtype=daft.DataType.list(daft.DataType.struct({"band": daft.DataType.int64(), "key": daft.DataType.int64()}))
)

# attach bands

df_bands = df_sig.with_column(
    "bands",
    udf_band(col("signature"), lit(R), lit(BANDS))
)

# explode to one row per (band, key)

df_pairs = df_bands.select("doc_id", col("bands").unnest().alias("pair")).select(
    "doc_id",
    col("pair").struct.get("band").alias("band"),
    col("pair").struct.get("key").alias("key"),
)

# group by (band, key) to collect candidate doc_ids

df_cands = df_pairs.groupby("band", "key").agg(col("doc_id").agg_list().alias("doc_ids"))

df_cands.show(5)



In [ ]:
# Aggregate clusters and preview

# Keep only buckets with at least 2 docs

df_clusters = df_cands.where(col("doc_ids").list.length() >= 2)

# Optional: compute a representative for each cluster (smallest doc_id)

df_clusters = df_clusters.with_column(
    "rep", col("doc_ids").list.sort().list.get(0)
)

# Explode to doc-level mapping
df_cluster_docs = df_clusters.select(
    "band", "key", "rep", col("doc_ids").unnest().alias("doc_id")
)

# Join back to optional metadata/text if desired
preview = df_cluster_docs.join(df.select("doc_id", "content_text"), on=["doc_id"])\
    .select("rep", "doc_id", "band", "key", "content_text")

preview.show(10)

